In [1]:
import sys

class ModuleMock:
    def __init__(self, module_name, classes_and_functions, parent=None):
        self.module_name = module_name
        self.classes_and_functions = classes_and_functions
        self.parent = parent
        self._set_submodules()
    
    def _set_submodules(self):
        if self.parent:
            full_module_name = f"{self.parent}.{self.module_name}"
        else:
            full_module_name = self.module_name
        
        for item in self.classes_and_functions:
            if item.startswith(full_module_name):
                parts = item.split('.')
                if len(parts) > (full_module_name.count('.') + 1):
                    submodule_name = parts[full_module_name.count('.') + 1]
                    if not hasattr(self, submodule_name):
                        setattr(self, submodule_name, ModuleMock(submodule_name, self.classes_and_functions, full_module_name))
    
    def __getattr__(self, name):
        full_name = f"{self.module_name}.{name}" if self.parent is None else f"{self.parent}.{self.module_name}.{name}"
        
        if full_name in self.classes_and_functions:
            def mock_function(*args, **kwargs):
                raise ImportError(
                    f"{self.module_name} is required for this method. "
                    f"Please install it using `pip install {self.module_name}`. "
                    f"For more information, check the requirements."
                )
            return mock_function
        else:
            raise AttributeError(f"Module '{self.module_name}' has no attribute '{name}'")

    def __call__(self, *args, **kwargs):
        raise ImportError(
            f"{self.module_name} is required for this method. "
            f"Please install it using `pip install {self.module_name}`. "
            f"For more information, check the requirements."
        )

def import_module_with_mock(module_name, classes_and_functions):
    try:
        return __import__(module_name)
    except ImportError:
        return ModuleMock(module_name, classes_and_functions)

# 유저가 제공하는 모듈 이름과 클래스/함수 목록
module_name = "torch"
classes_and_functions = ["torch.Tensor", "torch.nn.Module"]

# 모듈을 목킹하여 임포트
mocked_module = import_module_with_mock(module_name, classes_and_functions)

# mocked_module을 sys.modules에 추가하여 타입힌팅을 처리
sys.modules[module_name] = mocked_module

# 사용 예제
try:
    import torch
    tensor = torch.Tensor()
except ImportError as e:
    print(e)

try:
    module = torch.nn.Module()
except ImportError as e:
    print(e)


torch is required for this method. Please install it using `pip install torch`. For more information, check the requirements.
nn is required for this method. Please install it using `pip install nn`. For more information, check the requirements.


In [2]:
torch.classes_and_functions

['torch.Tensor', 'torch.nn.Module']

In [3]:
torch.parent

In [4]:

class Module:
    id = 'torch.nn.Module'
    pass

class nn:
    id = 'torch.nn'
    Module = Module()


class Tensor:
    id = 'torch.Tensor'
    pass

class torch:
    id = 'torch'
    Tensor = Tensor()
    nn = nn()

In [5]:
from typing import Dict, List

In [6]:
class DynamicModule:
    id:str
    name:str

In [7]:
full_ids = ["torch.nn.Module", "torch.Tensor"]

In [20]:
class DynamicModule:
    def __init__(self, name: str, id: str):
        self.name = name
        self.id = id

def create_class(class_name: str, attributes: Dict[str, str]) -> type:
    return type(class_name, (DynamicModule,), attributes)

def get_ids(full_path = "torch.nn.Module") -> List[str]:
    ids = []
    split = full_path.split('.')
    for i in range(len(split)):
        ids.append('.'.join(split[:i+1]))
    return ids

def create_modules(ids) -> Dict[str, DynamicModule]:
    modules = {}

    for id in ids:
        if id not in modules.keys():
            name = id.split('.')[-1]
            attributes = {
                'id': id,
                'name': name,
            }
            modules[id] = create_class(id, attributes)
    return modules

def collect_children(id, modules:Dict[str, DynamicModule])->List[DynamicModule]:
    children = []
    for candidate_id, module in modules.items():
        if candidate_id.startswith(id):
            parent_level = id.count('.')
            candidate_level = candidate_id.count('.')
            if candidate_level - parent_level == 1:
                children.append(module)
    return children

def set_children(parent, children:List[DynamicModule])->None:
    for child in children:    
        setattr(parent, child.name, child)

def generate_all_ids(full_ids)->List[str]:
    ids = []
    for full_id in full_ids:
        ids.extend(get_ids(full_id))
    return ids


def generate_module_complete(full_ids):
    ids = generate_all_ids(full_ids)

    modules = create_modules(ids)

    for id, module in modules.items():
        children = collect_children(id, modules)
        set_children(module, children)

    return modules

full_ids = ["torch.nn.Module", "torch.Tensor"]

modules = generate_module_complete(full_ids)

torch = modules['torch']

In [22]:
torch.nn.Module, torch.Tensor

(__main__.torch.nn.Module, __main__.torch.Tensor)

In [ ]:
def import_optional_module(module_name = 'torch'):
    return modules[module_name]

In [ ]:
try:
    import torch
except Exception as e:
    print(e)

In [ ]:
torch

In [14]:
from typing import Dict, List

class DynamicModule:
    def __init__(self, name: str, id: str):
        self.name = name
        self.id = id

def create_class(class_name: str, attributes: Dict[str, str]) -> type:
    return type(class_name, (DynamicModule,), attributes)

def get_ids(full_path: str) -> List[str]:
    ids = []
    split = full_path.split('.')
    for i in range(len(split)):
        ids.append('.'.join(split[:i+1]))
    return ids

def create_modules(ids: List[str]) -> Dict[str, DynamicModule]:
    modules = {}

    for id in ids:
        if id not in modules:
            name = id.split('.')[-1]
            attributes = {
                'id': id,
                'name': name,
            }
            modules[id] = create_class(name, attributes)
    return modules

def collect_children(id: str, modules: Dict[str, DynamicModule]) -> List[DynamicModule]:
    children = []
    for candidate_id, module in modules.items():
        if candidate_id.startswith(id):
            parent_level = id.count('.')
            candidate_level = candidate_id.count('.')
            if candidate_level - parent_level == 1:
                children.append(module)
    return children

def set_children(parent: DynamicModule, children: List[DynamicModule]) -> None:
    for child in children:
        setattr(parent, child.name, child)

def generate_all_ids(full_ids: List[str]) -> List[str]:
    ids = []
    for full_id in full_ids:
        ids.extend(get_ids(full_id))
    return ids

def generate_module_complete(full_ids: List[str]) -> Dict[str, DynamicModule]:
    ids = generate_all_ids(full_ids)

    modules = create_modules(ids)

    for id, module in modules.items():
        children = collect_children(id, modules)
        set_children(module, children)

    return modules

# 예시 사용법
full_ids = ["torch.nn.Module", "torch.Tensor"]
modules = generate_module_complete(full_ids)

torch = modules['torch']

# 동작 확인
print(torch.id)            # torch
print(torch.Tensor.id)     # torch.Tensor
print(torch.nn.Module.id)  # torch.nn.Module

# 기획된 에러
try:
    print(torch.Module)
except AttributeError as e:
    print(e)


torch
torch.Tensor
torch.nn.Module
type object 'torch' has no attribute 'Module'


In [17]:
torch.nn.Module

__main__.Module

In [18]:
from crimson.data_class import DataClassList

In [19]:
DataClassList

crimson.data_class.data_class_list.DataClassList

In [ ]:
import sys

class Generator():
    modules = {}

    @classmethod
    def create_module_structure(cls, module_name, children):
        
        
        



TypeError: 'DynamicModule' object is not callable

In [ ]:
import torch

In [ ]:
torch.nn

In [ ]:
torch.Tensor